In [2]:
import openai
import os
import sys
import termcolor
sys.path.append("../")
from magpie.prompt_planner.prompts import mp_prompt_thinker_coder_muk as mptc
from magpie.prompt_planner.prompts import dg_command_enumerator as dgce
from magpie.prompt_planner import conversation
from magpie.prompt_planner import confirmation_safe_executor
from magpie.prompt_planner import task_configs
import numpy as np
API_KEY = os.environ['CORRELL_API_KEY']

In [3]:
RESPONSE = '''

import sys
sys.path.append('../../')
sys.path.append('../')


def reset_reward(*args, **kwargs):
  pass


def execute_plan(*args, **kwargs):
  pass



from magpie.gripper import Gripper  # Import the gripper class
import numpy as np  # Import numpy for numerical operations

G = Gripper()  # Create a gripper object

# Since this is a new grasp, reset parameters to default to ensure no previous settings interfere.
G.reset_parameters()

# Set the goal aperture to 60.0 mm for the rubber duck which is small and lightweight.
goal_aperture = 28.0
# This is a complete grasp, so the gripper will not open after attempting the grasp.
complete_grasp = True
# Initial force calculation based on object weight and friction coefficient.
initial_force = (50 * 9.81) / (0.8 * 1000)  # 0.6125 N
# Additional closure if the gripper slips.
additional_closure = 2.0
# Additional force increase if the gripper slips, calculated as product of spring constant and additional closure.
additional_force = np.max([0.01, additional_closure * 100 * 0.0001])  # 0.02 N

# Move quickly to a safe aperture that is wider than the goal aperture to prepare for grasping.
G.set_goal_aperture(goal_aperture + additional_closure * 2, finger='both', record_load=False)

# Set compliance to allow a small margin of error in positioning and moderate flexibility.
G.set_compliance(1, 3, finger='both')

# Set the initial force to be applied by the gripper.
G.set_force(initial_force, 'both')

# Execute the grasp with the specified parameters.
G.deligrasp(goal_aperture, initial_force, additional_closure, additional_force, complete_grasp)

# Prediction of gripper's state after the commands:
# The gripper will attempt to close to the goal aperture of 60.0 mm, applying an initial force of 0.6125 N.
# If slipping is detected, it will close an additional 2.0 mm and increase the force by 0.02 N.
# Since the grasp is complete, the gripper will maintain its position and force after the adjustment.
'''

In [4]:
PROMPT_TYPE = 'thinker_coder'
TASK_CONFIG = task_configs.ALL_TASKS["magpie"]
openai.api_key = API_KEY
safe_executor = confirmation_safe_executor.ConfirmationSafeExecutor(
    skip_confirmation=True,
    local_execute=True)
PROMPT = TASK_CONFIG.prompts[PROMPT_TYPE]
PROMPT_MODEL = PROMPT(
    None, executor=safe_executor
)
pm = PROMPT_MODEL

In [5]:
stdout = pm.code_executor(RESPONSE)


ABOUT TO EXECUTE TC MUK 
 

import sys
sys.path.append('../../')
sys.path.append('../')


def reset_reward(*args, **kwargs):
  pass


def execute_plan(*args, **kwargs):
  pass



from magpie.gripper import Gripper  # Import the gripper class
import numpy as np  # Import numpy for numerical operations

G = Gripper()  # Create a gripper object

# Since this is a new grasp, reset parameters to default to ensure no previous settings interfere.
G.reset_parameters()

# Set the goal aperture to 60.0 mm for the rubber duck which is small and lightweight.
goal_aperture = 28.0
# This is a complete grasp, so the gripper will not open after attempting the grasp.
complete_grasp = True
# Initial force calculation based on object weight and friction coefficient.
initial_force = (50 * 9.81) / (0.8 * 1000)  # 0.6125 N
# Additional closure if the gripper slips.
additional_closure = 2.0
# Additional force increase if the gripper slips, calculated as product of spring constant and additional closure.
addi

In [6]:
import ast

# get content after last newline of stdout
sep = stdout.split('\n')
gl = np.array(ast.literal_eval(sep[-2]))
gl

array([{'timestamp': 1719530053.0437658, 'aperture': 27.80900566920235, 'gripper_vel': 14.440072235231236, 'contact_force': 0.08602071805555556, 'contact_force_l': 0.07723671388888889, 'contact_force_r': 0.09480472222222223, 'applied_force': 0.613125, 'k': 217.86995409540398},
       {'timestamp': 1719530053.2191775, 'aperture': 26.427462589527615, 'gripper_vel': 7.8760029701820775, 'contact_force': 0.11782769444444446, 'contact_force_l': 0.1539176888888889, 'contact_force_r': 0.08173770000000001, 'applied_force': 0.613125, 'k': 162.78403585375148},
       {'timestamp': 1719530053.4221268, 'aperture': 23.665361678302595, 'gripper_vel': 13.60980850128196, 'contact_force': 0.08450975535714286, 'contact_force_l': 0.052544228571428576, 'contact_force_r': 0.11647528214285714, 'applied_force': 0.633125, 'k': 233.42447227936773},
       {'timestamp': 1719530053.5965574, 'aperture': 22.05528107267126, 'gripper_vel': 9.230493530780441, 'contact_force': 0.10230953090277778, 'contact_force_l': 0.

In [16]:
# extract timestamps from gl
timestamps = []
for i in range(len(gl)):
    timestamps.append(gl[i]['timestamp'])
# 

In [21]:
import pandas as pd

# create dataframe from gl
df = pd.DataFrame(gl.tolist())

In [22]:
df


,timestamp,aperture,gripper_vel,contact_force,contact_force_l,contact_force_r,applied_force,k
0,1.719530e+09,27.809006,14.440072,0.086021,0.077237,0.094805,0.613125,217.869954
1,1.719530e+09,26.427463,7.876003,0.117828,0.153918,0.081738,0.613125,162.784036
2,1.719530e+09,23.665362,13.609809,0.084510,0.052544,0.116475,0.633125,233.424472
3,1.719530e+09,22.055281,9.230494,0.102310,0.135988,0.068631,0.633125,164.726591
4,1.719530e+09,20.218387,10.526343,0.094154,0.039821,0.148487,0.653125,172.950348
5,1.719530e+09,17.924664,12.100223,0.095947,0.083625,0.108270,0.653125,220.076399
6,1.719530e+09,16.781041,6.083376,0.129504,0.134823,0.124185,0.653125,148.103560
7,1.719530e+09,15.865747,4.200419,0.139259,0.065841,0.212677,0.673125,127.462927
8,1.719530e+09,15.865747,0.000000,0.241230,0.241820,0.240639,0.693125,0.000000


In [23]:
df[['timestamp', 'aperture', 'gripper_vel', 'contact_force']]


,timestamp,aperture,gripper_vel,contact_force
0,1.719530e+09,27.809006,14.440072,0.086021
1,1.719530e+09,26.427463,7.876003,0.117828
2,1.719530e+09,23.665362,13.609809,0.084510
3,1.719530e+09,22.055281,9.230494,0.102310
4,1.719530e+09,20.218387,10.526343,0.094154
5,1.719530e+09,17.924664,12.100223,0.095947
6,1.719530e+09,16.781041,6.083376,0.129504
7,1.719530e+09,15.865747,4.200419,0.139259
8,1.719530e+09,15.865747,0.000000,0.241230


In [26]:
df.iloc[0]['timestamp']

1719530053.0437658

In [37]:
import codecs
import json

def log_to_rlds():
    '''
    combine 3 logs:
    1. move (csv)
    2. grasp_log (json)
    3. home (csv)
    into one tfds in rlds format
    rlds: https://github.com/google-research/rlds?tab=readme-ov-file#load-with-tfds
    '''
    # list of dictionaries to tfds
    # https://stackoverflow.com/questions/68567630/converting-a-list-of-dictionaries-to-a-tf-dataset

    # load csv to pd df
    home = pd.read_csv("robot_logs/test.csv")
    move = pd.read_csv("robot_logs/test.csv")

    # extract timestamp columns and actual_qd columns from actual_qd_0 to actual_qd_5
    home = home[['timestamp', 'actual_qd_0', 'actual_qd_1', 'actual_qd_2', 'actual_qd_3', 'actual_qd_4', 'actual_qd_5']]
    move = move[['timestamp', 'actual_qd_0', 'actual_qd_1', 'actual_qd_2', 'actual_qd_3', 'actual_qd_4', 'actual_qd_5']]

    # load json
    path = "robot_logs/grasp_log.json"
    obj_text = codecs.open(path, 'r', encoding='utf-8').read()
    gl = np.array(json.loads(obj_text))
    # create df from np array of dictionaries with keys: timestamp, aperture, contact_force, and gripper_vel values
    grasp_log = pd.DataFrame(gl.tolist())
    # keep only timestamp, aperture, gripper_vel, and contact_force cols
    grasp_log = grasp_log[['timestamp', 'aperture', 'gripper_vel', 'contact_force']]
    # add actual_qd columns from actual_qd_0 to actual_qd_6 from the last row of move to grasp_log
    grasp_log[['actual_qd_0', 'actual_qd_1', 'actual_qd_2', 'actual_qd_3', 'actual_qd_4', 'actual_qd_5']] = move.iloc[-1][['actual_qd_0', 'actual_qd_1', 'actual_qd_2', 'actual_qd_3', 'actual_qd_4', 'actual_qd_5']]
    # add columns to move and home dfs, zeroed out
    move['aperture'] = 0
    move['gripper_vel'] = 0
    move['contact_force'] = 0
    home['aperture'] = 0
    home['gripper_vel'] = 0
    home['contact_force'] = 0


    # get initial values, iteratively subtract 0.1s from ti and assign to to move timestamps, from last to first
    ti = gl[0]['timestamp']
    ai = gl[0]['aperture']

    for i in np.arange(len(gl)-1, -1, -1):
        # iterate backwards through move timestamps
        ti -= 0.1
        move.loc[i, 'timestamp'] = ti
        move.loc[i, 'aperture'] = ai

    # get last timestamp tf, iteratively add 0.1s to tf and assign to to home timestamps, from first to last
    tf = gl[-1]['timestamp']
    af = gl[-1]['aperture']
    cf = gl[-1]['contact_force']

    for i in np.arange(len(home)):
        # iterate forwards through home timestamps
        tf += 0.1
        home.loc[i, 'timestamp'] = tf
        home.loc[i, 'aperture'] = af
        home.loc[i, 'contact_force'] = cf

    # combine dataframes
    df = pd.concat([move, grasp_log, home], axis=1)
    return df, grasp_log

df, gl = log_to_rlds()
df


/tmp/ipykernel_33451/632100740.py:33: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  grasp_log[['actual_qd_0', 'actual_qd_1', 'actual_qd_2', 'actual_qd_3', 'actual_qd_4', 'actual_qd_5']] = move.iloc[-1][['actual_qd_0', 'actual_qd_1', 'actual_qd_2', 'actual_qd_3', 'actual_qd_4', 'actual_qd_5']]
/tmp/ipykernel_33451/632100740.py:33: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  grasp_log[['actual_qd_0', 'actual_qd_1', 'actual_qd_2', 'actual_qd_3', 'actual_qd_4', 'actual_qd_5']] = move.iloc[-1][['actual_qd_0', 'actual_qd_1', 'actual_qd_2', 'actual_qd_3', 'actual_qd_4', 'actual_qd_5']]
/tmp/ipykernel_33451/632100740.py:33

,timestamp,actual_qd_0,actual_qd_1,actual_qd_2,actual_qd_3,actual_qd_4,actual_qd_5,aperture,gripper_vel,contact_force,...,timestamp,actual_qd_0,actual_qd_1,actual_qd_2,actual_qd_3,actual_qd_4,actual_qd_5,aperture,gripper_vel,contact_force
0,1.719530e+09,0.001078,-2.469845,2.349459,-1.955513,-1.612564,-3.034139,27.809006,0,0,...,1.719530e+09,0.001078,-2.469845,2.349459,-1.955513,-1.612564,-3.034139,15.865747,0,0.24123
1,1.719530e+09,0.001066,-2.469773,2.349483,-1.955452,-1.612540,-3.034175,27.809006,0,0,...,1.719530e+09,0.001066,-2.469773,2.349483,-1.955452,-1.612540,-3.034175,15.865747,0,0.24123
2,1.719530e+09,0.001006,-2.469785,2.349447,-1.955489,-1.612564,-3.034163,27.809006,0,0,...,1.719530e+09,0.001006,-2.469785,2.349447,-1.955489,-1.612564,-3.034163,15.865747,0,0.24123
3,1.719530e+09,0.001054,-2.469833,2.349519,-1.955452,-1.612564,-3.034163,27.809006,0,0,...,1.719530e+09,0.001054,-2.469833,2.349519,-1.955452,-1.612564,-3.034163,15.865747,0,0.24123
4,1.719530e+09,0.001054,-2.469821,2.349519,-1.955525,-1.612540,-3.034139,27.809006,0,0,...,1.719530e+09,0.001054,-2.469821,2.349519,-1.955525,-1.612540,-3.034139,15.865747,0,0.24123
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,2.077992e+06,-0.017217,-2.074527,2.649140,-2.651230,-1.607235,-3.047362,0.000000,0,0,...,1.719530e+09,-0.017217,-2.074527,2.649140,-2.651230,-1.607235,-3.047362,15.865747,0,0.24123
198,2.077992e+06,-0.017205,-2.074479,2.649140,-2.651218,-1.607187,-3.047386,0.000000,0,0,...,1.719530e+09,-0.017205,-2.074479,2.649140,-2.651218,-1.607187,-3.047386,15.865747,0,0.24123
199,2.077992e+06,-0.017229,-2.074467,2.649104,-2.651242,-1.607211,-3.047374,0.000000,0,0,...,1.719530e+09,-0.017229,-2.074467,2.649104,-2.651242,-1.607211,-3.047374,15.865747,0,0.24123
200,2.077992e+06,-0.017229,-2.074467,2.649164,-2.651230,-1.607235,-3.047386,0.000000,0,0,...,1.719530e+09,-0.017229,-2.074467,2.649164,-2.651230,-1.607235,-3.047386,15.865747,0,0.24123


In [8]:
import json
path = "robot_logs/grasp_logs.json"

# get content after last newline of stdout
with open(path, 'w') as f:
    json.dump(gl.tolist(), f)



In [9]:
import codecs
obj_text = codecs.open(path, 'r', encoding='utf-8').read()
b_new = json.loads(obj_text)
a_new = np.array(b_new)

In [2]:
task_configs.ALL_TASKS

{'magpie': TaskConfig(client=None, prompts={'thinker_coder': <class 'magpie.prompt_planner.prompts.mp_prompt_thinker_coder_muk.PromptThinkerCoder'>, 'thinker_coder_phys': <class 'magpie.prompt_planner.prompts.mp_prompt_tc_phys.PromptThinkerCoderPhys'>, 'thinker_coder_vision': <class 'magpie.prompt_planner.prompts.mp_prompt_tc_vision.PromptThinkerCoderVision'>, 'thinker_coder_vision_phys': <class 'magpie.prompt_planner.prompts.mp_prompt_tc_vision_phys.PromptThinkerCoderVisionPhys'>}),
 'system': TaskConfig(client=None, prompts={'system_executor': <class 'magpie.prompt_planner.prompts.dg_system_executor.PromptSystemExecutor'>}),
 'command_enumerator': TaskConfig(client=None, prompts={'command_enumerator': <class 'magpie.prompt_planner.prompts.dg_command_enumerator.PromptCommandEnumerator'>})}

In [3]:
tc = task_configs.ALL_TASKS['command_enumerator']
prompt = tc.prompts['command_enumerator']
MODEL = "gpt-3.5-turbo"
# MODEL = "gpt-4-turbo"
safe_executor = confirmation_safe_executor.ConfirmationSafeExecutor()
openai.api_key = API_KEY


In [8]:
# user_command = "gently pick the five largest and ripest tomatos and three small avocados"
# try:
#     prompt_model = prompt(
#         None, executor=safe_executor
#     )
#     conv = conversation.Conversation(prompt_model, MODEL)
#     # while True:
#       # user_command = input(termcolor.colored("User: ", "red", attrs=["bold"]))
#     try:
#       response = conv.send_command(user_command)
#       print(termcolor.colored("Magpie: ", "green", attrs=["bold"]) + response + "\n")
#     except Exception as e:  # pylint: disable=broad-exception-caught
#       print("Planning failed, try something else... " + str(e) + "\n")
#       # continue
    
#       # try:
#       #   prompt_model.code_executor(response)
#       # except Exception as e:  # pylint: disable=broad-exception-caught
#       #   print("Execution failed, try something else... " + str(e) + "\n")

# except Exception as e:
#     print(e)
#     prompt_model = None

LLM0 queried
[start of enumeration]
{'objects': ['tomato', 'avocado'],
'object_qualities': {'tomato': ['ripe', 'large'], 'avocado': ['small']},
'manipulation': 'pick',
'manipulation_qualities': ['gently']}
[end of enumeration]

processing response
processing command enumeration
LLM0 processed response: {'objects': ['tomato', 'avocado'],
'object_qualities': {'tomato': ['ripe', 'large'], 'avocado': ['small']},
'manipulation': 'pick',
'manipulation_qualities': ['gently']}
Magpie: {'objects': ['tomato', 'avocado'],
'object_qualities': {'tomato': ['ripe', 'large'], 'avocado': ['small']},
'manipulation': 'pick',
'manipulation_qualities': ['gently']}



In [11]:
import ast
ast.literal_eval(response)

{'objects': ['tomato', 'avocado'],
 'object_qualities': {'tomato': ['ripe', 'large'], 'avocado': ['small']},
 'manipulation': 'pick',
 'manipulation_qualities': ['gently']}

In [8]:
response = """
[start of enumeration]
1. Objects: ["avocados"]
2. Object Qualities: {"avocados": ["large"]}
3. Manipulation: "pick"
4. Manipulation Qualities: ["gently", "ignore irrelevant options"]
[end of enumeration]
"""

In [9]:
import re
import ast
import json
command_enumeration = (
    re.split(
        "end of enumeration",
        re.split("start of enumeration", response, flags=re.IGNORECASE)[
            1
        ],
        flags=re.IGNORECASE,
    )[0]
    .strip("[")
    .strip("]")
    .strip()
    .strip("```")
)


In [10]:
command_enumeration

'1. Objects: ["avocados"]\n2. Object Qualities: {"avocados": ["large"]}\n3. Manipulation: "pick"\n4. Manipulation Qualities: ["gently", "ignore irrelevant options"]'

In [11]:
# get object(s) as list from command_enumeration, try to use ast.literal_eval
objects = ast.literal_eval(command_enumeration.split("\n")[0].split(":")[1].strip())

In [28]:
oqual = command_enumeration.split("\n")[1]
start_index = oqual.find('{')
end_index = oqual.rfind('}')
oqual = ast.literal_eval(oqual[start_index:end_index+1])
oqual


In [32]:
# get manipulation type from command_enumeration as string
manipulation = ast.literal_eval(command_enumeration.split("\n")[2].split(":")[1].strip())
manipulation

'pick'

In [33]:
# get manipulation qualities from command_enumeration as list
mqual = ast.literal_eval(command_enumeration.split("\n")[3].split(":")[1].strip())
mqual

['gently', 'ignore irrelevant options']

In [13]:
object_qualities = ast.literal_eval(command_enumeration.split("\n")[1].split(":")[1].strip())

SyntaxError: '{' was never closed (<unknown>, line 1)

In [56]:
# get object(s) as list from command enumeration, after '1. Objects: ...' and before '2. Object Qualities: ...'
objects_search = re.search(r'1\. Objects:\s*\[(.*?)\]', command_enumeration)
if objects_search:
    objects = [obj.strip() for obj in objects_search.group(1).split(',')]
else:
    objects = []


In [57]:
objects

["'avocados'"]

In [58]:
qualities_match = re.search(r'2\. Object Qualities:\s*({.*?})', command_enumeration)
if qualities_match:
    object_qualities = json.loads(qualities_match.group(1).replace("'", '\"'))
else:
    object_qualities = {}

print("Object Qualities:", object_qualities)


Object Qualities: {'avocados': ['large']}


: 

In [32]:
tc = task_configs.ALL_TASKS['magpie']
tc.prompts['thinker_coder']


magpie.prompt_planner.prompts.mp_prompt_thinker_coder_muk.PromptThinkerCoder

In [6]:
import numpy as np
from PIL import Image
import base64
import io
import cv2 

def encode_image(img):
    # Convert NumPy array to base64 encoded string
    return base64.b64encode(img).decode('utf-8')


i = encode_image(np.array(Image.open("static/favicon.jpg")))

In [7]:
MODEL = "gpt-3.5-turbo"
import dataclasses
from typing import Any

In [8]:
@dataclasses.dataclass(frozen=True)
class TaskConfig:
  client: None
  prompts: dict[str, type[Any]]

ALL_TASKS = {
    'magpie': TaskConfig(
        client=None,
        prompts={
            'thinker_coder': mptc.PromptThinkerCoder,
        },
    ),
}


In [17]:
safe_executor = confirmation_safe_executor.ConfirmationSafeExecutor()
task_config = ALL_TASKS['magpie']
openai.api_key = API_KEY
prompt = task_config.prompts['thinker_coder']
MODEL = "gpt-3.5-turbo"

In [26]:
try:
    prompt_model = prompt(
        None, executor=safe_executor
    )
    conv = conversation.Conversation(prompt_model, MODEL)
    while True:
      user_command = input(termcolor.colored("User: ", "red", attrs=["bold"]))
      try:
        response = conv.send_command(user_command)
        print(termcolor.colored("Magpie: ", "green", attrs=["bold"]) + response + "\n")
      except Exception as e:  # pylint: disable=broad-exception-caught
        print("Planning failed, try something else... " + str(e) + "\n")
        continue
      
      try:
        prompt_model.code_executor(response)
      except Exception as e:  # pylint: disable=broad-exception-caught
        print("Execution failed, try something else... " + str(e) + "\n")

except Exception as e:
    print(e)
    prompt_model = None

LLM0 queried
[start of description]
* This is a new grasp.
* This grasp should be for picking up a grape.
* This is a complete grasp.
* This grasp is for an object with low compliance.
* The object has an approximate mass of 5 grams.
* The object has an approximate spring constant of 50 Newtons per meter.
* The gripper and object have an approximate friction coefficient of 0.2.
* This grasp should set the goal aperture to 5 mm.
* Based on object mass and friction coefficient, grasp should initially set the contact force to 1 Newton.
* If the gripper slips, this grasp should close an additional 1 mm.
* If the gripper slips, this grasp should increase the output force by 0.005 Newtons.
[end of description]

LLM1 queried
```python
from magpie.gripper import Gripper
G = Gripper()

import numpy as np

new_task = True

if new_task:
    G.reset_parameters()

goal_aperture = 5
complete_grasp = True
initial_force = (5 * 9.81) / 0.2
additional_closure = 1
spring_constant = 50
additional_force = 

KeyboardInterrupt: 